
---

# 🔧 What is a PATCH Request?

### 🧠 Simple Definition:

A **PATCH** request is used to **partially update** an existing resource — you send **only the fields that changed**, instead of the entire object.

> 🔁 Think of PUT as **replacing the whole document**, and PATCH as **editing just one line**.

---

## 🎯 Purpose of PATCH

| Use Case                         | Example                                 |
| -------------------------------- | --------------------------------------- |
| Update a single field            | Change only a patient’s weight          |
| Update a nested field            | Update only the emergency contact phone |
| Optimize network payloads        | Avoid sending unchanged fields          |
| Avoid overwriting default values | Prevent nulls or resetting fields       |

---

# 🔂 PUT vs PATCH vs POST

| Feature         | POST | PUT               | PATCH                |
| --------------- | ---- | ----------------- | -------------------- |
| Create?         | ✅    | ❌ (update only)   | ❌                    |
| Replace?        | ❌    | ✅ (entire object) | ❌ (partial)          |
| Update Partial? | ❌    | ❌                 | ✅                    |
| Idempotent?     | ❌    | ✅                 | ✅ (if used properly) |

---

## 🏥 Scenario: Update Only Patient’s Weight (Don’t touch anything else)

You’re managing a patient:

```json
{
  "id": 101,
  "name": "Mahbub Hossain",
  "age": 28,
  "weight_kg": 72,
  "height_cm": 170
}
```

The patient’s weight changed. But you **don’t** want to resend name, age, height.

---

# ✅ FastAPI PATCH Code Example

We’ll follow these steps:

1. Define a **partial model** using `Optional[...]`
2. Get existing data from DB
3. Merge updates
4. Validate and recompute computed fields

---

## ✅ Step-by-Step Code

```python
from fastapi import FastAPI, Path
from pydantic import BaseModel, Field, computed_field
from typing import Optional

app = FastAPI()

# Simulated DB
mock_db = {
    101: {
        "name": "Mahbub Hossain",
        "age": 28,
        "weight_kg": 72,
        "height_cm": 170
    }
}

# Full model
class Patient(BaseModel):
    name: str
    age: int
    weight_kg: float
    height_cm: float

    @computed_field
    @property
    def bmi(self) -> float:
        h = self.height_cm / 100
        return round(self.weight_kg / (h ** 2), 2)

# ✅ PATCH model: all fields optional
class PartialPatient(BaseModel):
    name: Optional[str] = None
    age: Optional[int] = None
    weight_kg: Optional[float] = None
    height_cm: Optional[float] = None

# PATCH endpoint
@app.patch("/patients/{patient_id}")
def patch_patient(patient_id: int, update: PartialPatient):
    if patient_id not in mock_db:
        return {"error": f"Patient {patient_id} not found"}

    # Step 1: Load current patient data
    current_data = mock_db[patient_id]

    # Step 2: Merge updates
    updated_dict = {**current_data, **update.model_dump(exclude_unset=True)}

    # Step 3: Re-validate full model + recompute BMI
    updated_patient = Patient(**updated_dict)

    # Step 4: Save to DB
    mock_db[patient_id] = updated_patient.model_dump()

    return {
        "message": f"Patient {patient_id} partially updated!",
        "bmi": updated_patient.bmi,
        "data": updated_patient.model_dump()
    }
```

---

## 🧪 Sample PATCH Request

**Endpoint:** `PATCH /patients/101`

**Body:**

```json
{
  "weight_kg": 78
}
```

---

### ✅ Output:

```json
{
  "message": "Patient 101 partially updated!",
  "bmi": 26.99,
  "data": {
    "name": "Mahbub Hossain",
    "age": 28,
    "weight_kg": 78.0,
    "height_cm": 170.0
  }
}
```

> 🎯 BMI is recomputed. Only the changed field was sent.

---